<a href="https://colab.research.google.com/github/navi004/Deep-Learning-CSE4037-/blob/main/DL_Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LSTM and GRU - Lab A-5.1
* Naveen N (22MIA1049)  

In [1]:
# Install kagglehub
!pip install -q kagglehub

# Import kagglehub
import kagglehub

# Download the latest version of the IMDB dataset
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:00<00:00, 43.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [2]:
import os

# Check the files in the path where the dataset is downloaded
print("Files in the dataset directory:")
os.listdir(path)

Files in the dataset directory:


['IMDB Dataset.csv']

In [42]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM,GRU, Dense
from sklearn.model_selection import train_test_split

In [4]:
data_file = os.path.join(path, 'IMDB Dataset.csv')
df = pd.read_csv(data_file)
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [31]:
t = Tokenizer(num_words=5000)

t.fit_on_texts(df['review'])

x = t.texts_to_sequences(df['review'])

max_length = 100

xp = pad_sequences(x, maxlen=max_length, padding = 'post')

In [32]:
xp

array([[ 954,  183,    1, ...,  125, 4103,  486],
       [  29,    1, 2294, ..., 1977,   69,  221],
       [1026,  220,  230, ...,   63,   16,  350],
       ...,
       [   9,  200,   26, ..., 1957,    2,    2],
       [  16,   47, 4834, ...,   67,  739,   42],
       [  49,   14,   47, ...,  794,   11,   17]], dtype=int32)

In [33]:
y = np.array(df['sentiment'].map({'positive' : 1, 'negative' : 0}))
y

array([1, 1, 1, ..., 0, 0, 0])

In [34]:
x_train, x_test, y_train, y_test = train_test_split(xp, y, test_size=0.2, random_state=42)

print(x_train.shape)

(40000, 100)


### LSTM Model

In [35]:
lstm_model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length = max_length),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 55s 42ms/step - accuracy: 0.7425 - loss: 0.4962 - val_accuracy: 0.8599 - val_loss: 0.3259
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 41ms/step - accuracy: 0.8660 - loss: 0.3286 - val_accuracy: 0.8573 - val_loss: 0.3371
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.8894 - loss: 0.2738 - val_accuracy: 0.8662 - val_loss: 0.3118
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 40ms/step - accuracy: 0.9109 - loss: 0.2272 - val_accuracy: 0.8637 - val_loss: 0.3143
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - accuracy: 0.9182 - loss: 0.2076 - val_accuracy: 0.8549 - val_loss: 0.3530


### Accuracy and loss

In [37]:
lstm_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (32, 100, 32)               │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (32, 32)                    │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (32, 1)                     │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 505,061 (1.93 MB)

 Trainable params: 168,353 (657.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 336,708 (1.28 MB)

In [36]:
loss, accuracy = lstm_model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8562 - loss: 0.3540
Test Loss: 0.3530319929122925, Test Accuracy: 0.8549000024795532


### Testing with new record

In [55]:
txt = 'The movie is so boring and the direction is bad'

txt = t.texts_to_sequences([txt])

txtP = pad_sequences(txt, maxlen=max_length, padding='post')

pred_lstm = lstm_model.predict(txtP)

print(pred_lstm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0.06063591]]


In [58]:
print("Sentiment : Positive" if pred_lstm[0][0] > 0.5 else "Sentiment : Negative")

Sentiment : Negative


### GRU

In [43]:
gru_model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length = max_length),
    GRU(32),
    Dense(1, activation='sigmoid')
])

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru_history = gru_model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 61s 47ms/step - accuracy: 0.7078 - loss: 0.5274 - val_accuracy: 0.8600 - val_loss: 0.3226
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 57s 45ms/step - accuracy: 0.8871 - loss: 0.2799 - val_accuracy: 0.8738 - val_loss: 0.2920
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 59s 47ms/step - accuracy: 0.9073 - loss: 0.2390 - val_accuracy: 0.8753 - val_loss: 0.2917
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 47ms/step - accuracy: 0.9247 - loss: 0.1997 - val_accuracy: 0.8690 - val_loss: 0.3011
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 62s 49ms/step - accuracy: 0.9391 - loss: 0.1669 - val_accuracy: 0.8661 - val_loss: 0.3234


In [46]:
gru_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (32, 100, 32)               │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (32, 32)                    │           6,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (32, 1)                     │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 499,109 (1.90 MB)

 Trainable params: 166,369 (649.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 332,740 (1.27 MB)

In [48]:
test_loss, test_accuracy = gru_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8625 - loss: 0.3232
Test Loss: 0.3234159052371979, Test Accuracy: 0.866100013256073


### Testing with new review

In [59]:
txt = 'The movie is very good and i love the story and direction, it got very good twists'

txt = t.texts_to_sequences([txt])

txtP = pad_sequences(txt, maxlen=max_length, padding='post')

pred_gru = lstm_model.predict(txtP)

print(pred_gru)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[[0.9737275]]


In [61]:
print("Sentiment : Positive" if pred_gru[0][0] > 0.5 else "Sentiment : Negative")

Sentiment : Positive
